# MT5 LSTM Forward Testing Notebook
# Timeframe: M5
# Prediction horizon: 25 minutes (5 candles)
# Risk-Reward: 1:2

In [1]:
#Imports

In [2]:
#D!pip install MetaTrader5

In [3]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
#MT5 Initialization
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M5
MAGIC = 250925
LOT = 0.01


if not mt5.initialize():
     raise RuntimeError(mt5.last_error())


mt5.symbol_select(SYMBOL, True)
print("MT5 initialized")

MT5 initialized


In [5]:
#Pull 1 Week of Historical Data (Training)
end = datetime.now()
start = end - timedelta(days=7)


rates = mt5.copy_rates_range(SYMBOL, TIMEFRAME, start, end)


df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')

In [6]:
#Feature Engineering (MATCH BACKTEST)
df['return'] = df['close'].pct_change()
df['target'] = df['close'].shift(-5) # 25 minutes ahead


df.dropna(inplace=True)


features = ['open','high','low','close','tick_volume','return']

In [7]:
#Scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[features])

In [8]:
#Sequence Builder
LOOKBACK = 50


X, y = [], []
for i in range(LOOKBACK, len(df)):
    X.append(scaled_features[i-LOOKBACK:i])
    y.append(1 if df['target'].iloc[i] > df['close'].iloc[i] else 0)


X, y = np.array(X), np.array(y)

In [9]:
#LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\dell\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
#Training (ES + MC)
es = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,
    verbose=1
)


mc = ModelCheckpoint(
    'best_model_LSTM_GOLD.keras',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True
)


model.fit(
    X, y,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[es, mc],
    shuffle=False
)


model = load_model('best_model_LSTM_GOLD.keras')

Epoch 1/50
31/33 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4375 - loss: 0.6981
Epoch 1: val_loss improved from None to 0.69139, saving model to best_model_LSTM_GOLD.keras
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.4451 - loss: 0.6983 - val_accuracy: 0.5878 - val_loss: 0.6914
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5298 - loss: 0.6953
Epoch 2: val_loss improved from 0.69139 to 0.67932, saving model to best_model_LSTM_GOLD.keras
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5358 - loss: 0.6947 - val_accuracy: 0.5992 - val_loss: 0.6793
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5311 - loss: 0.6935
Epoch 3: val_loss improved from 0.67932 to 0.67747, saving model to best_model_LSTM_GOLD.keras
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5396 - loss: 0.6931 - val_accuracy: 0.5992 - val_loss: 0.6775
Epoch 4/50
32/33 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5305 - loss: 0.6933
Epoch 4: val_loss improved fro

In [11]:
#ATR for SL/TP
def ATR(data, period=14):
    high_low = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close = np.abs(data['low'] - data['close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    return true_range.rolling(period).mean().iloc[-1]

In [12]:
######################3


In [13]:
#only one trade at a time/ for up alows change in direction
#Trade Functions 123
#STRONG_BUY  = 0.65
#STRONG_SELL = 0.35
STRONG_BUY  = 0.6
STRONG_SELL = 0.4

last_signal = None 

def has_position():
    positions = mt5.positions_get(symbol=SYMBOL)
    return positions is not None and len(positions) > 0
    



def send_trade(signal, atr):
    price = mt5.symbol_info_tick(SYMBOL).ask if signal == 1 else mt5.symbol_info_tick(SYMBOL).bid 
    sl = price - atr if signal == 1 else price + atr 
    tp = price + (2 * atr) if signal == 1 else price - (2 * atr)


    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT,
        "type": mt5.ORDER_TYPE_BUY if signal == 1 else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "magic": MAGIC,
        "comment": "LSTM_25min",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }


    result = mt5.order_send(request)
    print(result)



In [ ]:
print("Starting forward testing...")

last_bar_time = None
last_signal = None

while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')

    current_bar_time = df_live['time'].iloc[-1]

    # Act only on NEW candle
    if last_bar_time == current_bar_time:
        time.sleep(10)
        continue

    last_bar_time = current_bar_time

    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)

    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)

    prediction = model.predict(X_live, verbose=0)[0][0]

    # ---- STRONG SIGNAL FILTER ----
    if prediction > STRONG_BUY:
        signal = 1
        signal_txt = "STRONG BUY"
    elif prediction < STRONG_SELL:
        signal = 0
        signal_txt = "STRONG SELL"
    else:
        signal = None
        signal_txt = "NO TRADE"

    print(
        f"[{current_bar_time}] "
        f"Pred={prediction:.4f} | "
        f"Signal={signal_txt} | "
        f"HasPosition={has_position()}"
    )

    # ---- ENTRY LOGIC ----
    if signal is not None and not has_position():

        if last_signal == signal:
            print("  ↳ Skipped (same direction as last trade)")
            continue

        atr = ATR(df_live)
        print(f"  ↳ Sending trade | ATR={atr:.2f}")

        send_trade(signal, atr)
        last_signal = signal

    time.sleep(10)


Starting forward testing...
[2025-12-23 18:00:00] Pred=0.5738 | Signal=NO TRADE | HasPosition=False


In [ ]:
#Forward Testing Loop 123
print("Starting forward testing...")


last_bar_time = None
last_signal = None

while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')

    current_bar_time = df_live['time'].iloc[-1]
    
    if last_bar_time == current_bar_time :
        time.sleep(10)
        continue
    
    
    last_bar_time = current_bar_time
    
    
    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)
    
    
    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)
    
    
    prediction = model.predict(X_live, verbose=0)[0][0]
    # ---- STRONG SIGNAL FILTER ----
    if prediction > STRONG_BUY:
        signal = 1
    elif prediction < STRONG_SELL:
        signal = 0
    else:
        signal = None   # weak / no trade

    # ---- ENTRY LOGIC ----
    if signal is not None and not has_position():
        # prevent repeated same-direction entries
        if last_signal == signal:
             time.sleep(10)
             continue
    
        atr = ATR(df_live)
        send_trade(signal, atr)
        last_signal = signal   # remember last trade direction
    
        time.sleep(10)

In [ ]:
#Trade Functions
def get_open_position():
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions is None:
      return None
    for p in positions:
        if p.magic == MAGIC:
          return p
    return None




def close_position(position):
    price = mt5.symbol_info_tick(SYMBOL).bid if position.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": SYMBOL,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
        "price": price,
        "magic": MAGIC,
        "comment": "LSTM_early_exit",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    mt5.order_send(request)




def send_trade(signal, atr):
    price = mt5.symbol_info_tick(SYMBOL).ask if signal == 1 else mt5.symbol_info_tick(SYMBOL).bid
    sl = price - atr if signal == 1 else price + atr
    tp = price + (2 * atr) if signal == 1 else price - (2 * atr)
    
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT,
        "type": mt5.ORDER_TYPE_BUY if signal == 1 else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "magic": MAGIC,
        "comment": "LSTM_25min",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    
    mt5.order_send(request)
    print(result)

In [ ]:
# CELL 11 — Forward Testing Loop (FIXED: one trade per candle)
# =========================
print("Starting forward testing...")


last_processed_bar_time = None


while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')
    
    
    current_bar_time = df_live['time'].iloc[-1]
    
    
    # Act only on NEW candle close
    if last_processed_bar_time == current_bar_time:
        time.sleep(10)
        continue
    
    
    last_processed_bar_time = current_bar_time
    
    
    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)
    
    
    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)
    
    
    prediction = model.predict(X_live, verbose=0)[0][0]
    
    
    # Strong signal thresholds
    STRONG_BUY = 0.65
    STRONG_SELL = 0.35
    
    
    if prediction > STRONG_BUY:
        signal = 1
    elif prediction < STRONG_SELL:
        signal = 0
    else:
        signal = None
    
    
    position = get_open_position()
    
    
    # Early exit on strong opposite signal
    if position is not None:
        if position.type == mt5.ORDER_TYPE_BUY and prediction < STRONG_SELL:
            print("Strong opposite SELL — closing BUY")
            close_position(position)
            time.sleep(5)
            continue
        elif position.type == mt5.ORDER_TYPE_SELL and prediction > STRONG_BUY:
            print("Strong opposite BUY — closing SELL")
            close_position(position)
            time.sleep(5)
            continue
        
    
    # Entry logic — one trade per candle
    if position is None and signal is not None:
        atr = ATR(df_live)
        send_trade(signal, atr)
    
    
    time.sleep(10)

In [ ]:
#Forward Testing Loop
print("Starting forward testing...")


last_bar_time = None


while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')
    
    
    if last_bar_time == df_live['time'].iloc[-1]:
        time.sleep(10)
        continue
        
    
    last_bar_time = df_live['time'].iloc[-1]
    
    
    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)
    
    
    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)


    prediction = model.predict(X_live)[0][0]


# Signal strength thresholds
STRONG_BUY = 0.65
STRONG_SELL = 0.35


signal = 1 if prediction > 0.5 else 0


position = get_open_position()


    # Early exit on strong opposite signal
if position is not None:
    if position.type == mt5.ORDER_TYPE_BUY and prediction < STRONG_SELL:
        print("Strong opposite SELL signal detected — closing BUY")
        close_position(position)
    elif position.type == mt5.ORDER_TYPE_SELL and prediction > STRONG_BUY:
        print("Strong opposite BUY signal detected — closing SELL")
        close_position(position)
        
    
# Entry logic (one trade at a time)
if position is None:
    atr = ATR(df_live)
    send_trade(signal, atr)


# Sleep until next check
time.sleep(60)

In [ ]:
#only one trade at a time/ for up alows change in direction
#Trade Functions 123
def has_position():
    positions = mt5.positions_get(symbol=SYMBOL)
    return positions is not None and len(positions) > 0
    



def send_trade(signal, atr):
    price = mt5.symbol_info_tick(SYMBOL).ask if signal == 1 else mt5.symbol_info_tick(SYMBOL).bid 
    sl = price - atr if signal == 1 else price + atr 
    tp = price + (2 * atr) if signal == 1 else price - (2 * atr)


    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT,
        "type": mt5.ORDER_TYPE_BUY if signal == 1 else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "magic": MAGIC,
        "comment": "LSTM_25min",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }


    result = mt5.order_send(request)
    print(result)



In [ ]:
#Forward Testing Loop 123
print("Starting forward testing...")


last_bar_time = None


while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')
    
    
    if last_bar_time == df_live['time'].iloc[-1]:
        time.sleep(10)
        continue
    
    
    last_bar_time = df_live['time'].iloc[-1]
    
    
    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)
    
    
    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)
    
    
    prediction = model.predict(X_live)[0][0]
    signal = 1 if prediction > 0.5 else 0
    
    
    if not has_position():
        atr = ATR(df_live)
        send_trade(signal, atr)
    
    
    time.sleep(60)

In [ ]:
# MT5 LSTM Forward Testing Notebook
# Timeframe: M5
# Prediction horizon: 25 minutes (5 candles)
# Risk-Reward: 1:2

# =========================
# CELL 1 — Imports
# =========================
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# =========================
# CELL 2 — MT5 Initialization
# =========================
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M5
MAGIC = 250925
LOT = 0.01

if not mt5.initialize():
    raise RuntimeError(mt5.last_error())

mt5.symbol_select(SYMBOL, True)
print("MT5 initialized")

# =========================
# CELL 3 — Pull 1 Week of Historical Data (Training)
# =========================
end = datetime.now()
start = end - timedelta(days=7)

rates = mt5.copy_rates_range(SYMBOL, TIMEFRAME, start, end)

df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')

# =========================
# CELL 4 — Feature Engineering (MATCH BACKTEST)
# =========================
df['return'] = df['close'].pct_change()
df['target'] = df['close'].shift(-5)  # 25 minutes ahead

df.dropna(inplace=True)

features = ['open','high','low','close','tick_volume','return']

# =========================
# CELL 5 — Scaling
# =========================
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[features])

# =========================
# CELL 6 — Sequence Builder
# =========================
LOOKBACK = 50

X, y = [], []
for i in range(LOOKBACK, len(df)):
    X.append(scaled_features[i-LOOKBACK:i])
    y.append(1 if df['target'].iloc[i] > df['close'].iloc[i] else 0)

X, y = np.array(X), np.array(y)

# =========================
# CELL 7 — LSTM Model
# =========================
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# =========================
# =========================
# CELL 8 — Training (ES + MC) — MATCH BACKTEST
# =========================
es = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,
    verbose=1
)

mc = ModelCheckpoint(
    'best_model_LSTM_GOLD.keras',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True
)

model.fit(
    X, y,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[es, mc],
    shuffle=False
)

model = load_model('best_model_LSTM_GOLD.keras')

# =========================
# CELL 9 — ATR for SL/TP
# =========================
def ATR(data, period=14):
    high_low = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close = np.abs(data['low'] - data['close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    return true_range.rolling(period).mean().iloc[-1]

# =========================
# CELL 10 — Trade Functions
# =========================
def get_open_position():
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions is None:
        return None
    for p in positions:
        if p.magic == MAGIC:
            return p
    return None


def close_position(position):
    price = mt5.symbol_info_tick(SYMBOL).bid if position.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": SYMBOL,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
        "price": price,
        "magic": MAGIC,
        "comment": "LSTM_early_exit",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    mt5.order_send(request)


def send_trade(signal, atr):
    price = mt5.symbol_info_tick(SYMBOL).ask if signal == 1 else mt5.symbol_info_tick(SYMBOL).bid
    sl = price - atr if signal == 1 else price + atr
    tp = price + (2 * atr) if signal == 1 else price - (2 * atr)

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT,
        "type": mt5.ORDER_TYPE_BUY if signal == 1 else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "magic": MAGIC,
        "comment": "LSTM_25min",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    mt5.order_send(request)

# =========================
# CELL 11 — Forward Testing Loop (FIXED: one trade per candle)
# =========================
print("Starting forward testing...")

last_processed_bar_time = None

while True:
    rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, LOOKBACK + 20)
    df_live = pd.DataFrame(rates)
    df_live['time'] = pd.to_datetime(df_live['time'], unit='s')

    current_bar_time = df_live['time'].iloc[-1]

    # Act only on NEW candle close
    if last_processed_bar_time == current_bar_time:
        time.sleep(10)
        continue

    last_processed_bar_time = current_bar_time

    df_live['return'] = df_live['close'].pct_change()
    df_live.dropna(inplace=True)

    X_live = scaler.transform(df_live[features])[-LOOKBACK:]
    X_live = np.expand_dims(X_live, axis=0)

    prediction = model.predict(X_live, verbose=0)[0][0]

    # Strong signal thresholds
    STRONG_BUY = 0.65
    STRONG_SELL = 0.35

    if prediction > STRONG_BUY:
        signal = 1
    elif prediction < STRONG_SELL:
        signal = 0
    else:
        signal = None

    position = get_open_position()

    # Early exit on strong opposite signal
    if position is not None:
        if position.type == mt5.ORDER_TYPE_BUY and prediction < STRONG_SELL:
            print("Strong opposite SELL — closing BUY")
            close_position(position)
            time.sleep(5)
            continue
        elif position.type == mt5.ORDER_TYPE_SELL and prediction > STRONG_BUY:
            print("Strong opposite BUY — closing SELL")
            close_position(position)
            time.sleep(5)
            continue

    # Entry logic — one trade per candle
    if position is None and signal is not None:
        atr = ATR(df_live)
        send_trade(signal, atr)

    time.sleep(10)
